# Reports

## Schema

### Setup

In [1]:
#!fsharp
#r "nuget: FSharp.Data"
#r "nuget: FsToolkit.ErrorHandling"
#r "./src/Reports/bin/Debug/net5.0/Reports.dll"

open FSharp.Data
open OSTOR.ClinicalTrials.Reports.Utilities
open OSTOR.ClinicalTrials.Reports.Utilities.Xml

open System
open System.IO

Installed package FsToolkit.ErrorHandling version 2.0.0

Installed package FSharp.Data version 3.3.3

### Caris Schema

In [1]:
#!fsharp
let carisSchemaGenerator = XmlSchemaGenerator()

// Caris report xmls
let carisReportXmls = Path.Combine([|Environment.CurrentDirectory; "data"; "Caris"|])

let sg =
    DirectoryInfo(carisReportXmls).EnumerateFileSystemInfos("*.xml")
        |> Seq.fold (fun (sg: XmlSchemaGenerator) fileInfo -> sg.AddXml(fileInfo.FullName)) carisSchemaGenerator

sg.SaveSchemaToDisk "./src/Reports/data/carisReport.xsd"

## Validations

### Setup

In [1]:
#!fsharp
#r "nuget: FSharp.Data"
#r "nuget: FsToolkit.ErrorHandling"
#r "./src/Reports/bin/Debug/net5.0/Reports.dll"

open OSTOR.ClinicalTrials.Reports

open System
open System.IO

### FMI Validations

In [1]:
#!fsharp
#r "nuget: FSharp.Data"
#r "nuget: FsToolkit.ErrorHandling"
#r "./src/Reports/bin/Debug/net5.0/Reports.dll"

open OSTOR.ClinicalTrials.Reports

open System
open System.IO

open Utilities

let fmiReportXmls = Path.Combine([|Environment.CurrentDirectory; "data"; "FMI"|])

let fmiResults =
    DirectoryInfo(fmiReportXmls).EnumerateFileSystemInfos("*.xml")
    |> Seq.map (fun filePath -> FoundationMedicine.Report.Xml(filePath.FullName).ReportInput |> FoundationMedicine.Report.validate)

let (reports, errors) = fmiResults |> Seq.toList |> Result.partition

#### Errors

In [1]:
#!fsharp
errors

(empty)